In [3]:
from dotenv import load_dotenv

_ = load_dotenv()

In [4]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langgraph.checkpoint.sqlite import SqliteSaver

### Define a simple 2 node graph with the following state: -lnode: last node -scratch: a scratchpad location -count : a counter that is incremented each step

In [5]:
class AgentState(TypedDict):
    lnode: str
    scratch: str
    count: Annotated[int, operator.add]

In [6]:
def node1(state: AgentState):
    print(f"node1, count:{state['count']}")
    return {"lnode": "node_1",
            "count": 1,
            }

def node2(state: AgentState):
    print(f"node2, count:{state['count']}")
    return {"lnode": "node_2",
            "count": 1,
            }

In [7]:
def should_continue(state):
    return state["count"] < 3

In [8]:
builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)

builder.add_edge("Node1", "Node2")
builder.add_conditional_edges("Node2", 
                              should_continue, 
                              {True: "Node1", False: END})
builder.set_entry_point("Node1")

In [9]:
memory = SqliteSaver.from_conn_string(":memory:")
graph = builder.compile(checkpointer=memory)

In [10]:
thread = {"configurable": {"thread_id": str(1)}}
graph.invoke({"count":0, "scratch":"hi"},thread)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [11]:
graph.get_state(thread)

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-a001-68b4-8004-754f2521f931'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'count': 1, 'lnode': 'node_2'}}}, created_at='2024-06-07T17:52:37.289797+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9ffc-6ae1-8003-ea5a6b41450f'}})

In [ ]:
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-a001-68b4-8004-754f2521f931'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'count': 1, 'lnode': 'node_2'}}}, created_at='2024-06-07T17:52:37.289797+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9ffc-6ae1-8003-ea5a6b41450f'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9ffc-6ae1-8003-ea5a6b41450f'}}, metadata={'source': 'loop', 'step': 3, 'writes': {'Node1': {'count': 1, 'lnode': 'node_1'}}}, created_at='2024-06-07T17:52:37.287804+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9ff8-6e15-8002-f88d73188302'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 2}, next=('Node1',), config={'configurable': {'thread_id': '1', 'thread_t

In [13]:
states = []
for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values['count'])

{'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-a001-68b4-8004-754f2521f931'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9ffc-6ae1-8003-ea5a6b41450f'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9ff8-6e15-8002-f88d73188302'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9ff0-6cfa-8001-a1b4f12d6f5b'}} 1
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9fdd-6482-8000-c2dd444592a9'}} 0
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9fd8-6661-bfff-03b463037116'}} 0


In [14]:
states[-3]

{'configurable': {'thread_id': '1',
  'thread_ts': '1ef24f6b-9ff0-6cfa-8001-a1b4f12d6f5b'}}

In [15]:
graph.get_state(states[-3])

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9ff0-6cfa-8001-a1b4f12d6f5b'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'Node1': {'count': 1, 'lnode': 'node_1'}}}, created_at='2024-06-07T17:52:37.282943+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9fdd-6482-8000-c2dd444592a9'}})

In [16]:
graph.invoke(None, states[-3])

node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [17]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state.config, state.values['count'])

{'configurable': {'thread_id': '1', 'thread_ts': '1ef24fa3-d837-6dba-8004-0967845738d3'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24fa3-d82d-63c0-8003-354f11d9d982'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24fa3-d823-691b-8002-5872cf92aaed'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-a001-68b4-8004-754f2521f931'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9ffc-6ae1-8003-ea5a6b41450f'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9ff8-6e15-8002-f88d73188302'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9ff0-6cfa-8001-a1b4f12d6f5b'}} 1
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9fdd-6482-8000-c2dd444592a9'}} 0
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24f6b-9fd8-6661-bfff-03b463037116'}} 0


In [18]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state,"\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24fa3-d837-6dba-8004-0967845738d3'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'count': 1, 'lnode': 'node_2'}}}, created_at='2024-06-07T18:17:46.422623+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24fa3-d82d-63c0-8003-354f11d9d982'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24fa3-d82d-63c0-8003-354f11d9d982'}}, metadata={'source': 'loop', 'step': 3, 'writes': {'Node1': {'count': 1, 'lnode': 'node_1'}}}, created_at='2024-06-07T18:17:46.418272+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24fa3-d823-691b-8002-5872cf92aaed'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 2}, next=('Node1',), config={'configurable': {'thread_id': '1', 'thread_t